In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [0]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [0]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

In [0]:
def createmodel():
    embed_dim= 128
    lstm_out = 196
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [8]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 25, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
 - 25s - loss: 0.8261 - accuracy: 0.6445
Epoch 2/25
 - 24s - loss: 0.6874 - accuracy: 0.7025
Epoch 3/25
 - 24s - loss: 0.6215 - accuracy: 0.7387
Epoch 4/25
 - 24s - loss: 0.5735 - accuracy: 0.7600
Epoch 5/25
 - 24s - loss: 0.5372 - accuracy: 0.7734
Epoch 6/25
 - 24s - loss: 0.4959 - accuracy: 0.7967
Epoch 7/25
 - 24s - loss: 0.4578 - accuracy: 0.8118
Epoch 8/25
 - 24s - loss: 0.4235 - accuracy: 0.8297
Epoch 9/25
 - 24s - loss: 0.3978 - accuracy: 0.8414
Epoch 10/25
 - 25s - loss: 0.3736 - accuracy: 0.8503
Epoch 11/25
 - 25s - loss: 0.3494 - accuracy: 0.8548
Epoch 12/25
 - 24s - loss: 0.3293 - accuracy: 0.8666
Epoch 13/25
 - 25s - loss: 0.3168 - accuracy: 0.8726
Epoch 14/25
 - 25s - loss: 0.2978 - accuracy: 0.8800
Epoch 15/25
 - 24s - loss: 0.2832 - accuracy: 0.8865
Epoch 16/25
 - 24s - loss: 0.2734 - accuracy: 0.8886
Epoch 17/25
 - 24s - loss: 0.2592 - accuracy: 0.8930
Epoch 18/25
 - 24s - loss: 0.2546 - accuracy: 0.8953
Epoch 19/25
 - 24s - loss: 0.2436 - accuracy: 0.9020
Ep

In [0]:
model.save('sentimentAnalysis.h5')

In [10]:
from keras.models import load_model
model= load_model('sentimentAnalysis.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [12]:
import numpy as np
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing']
sentence = tokenizer.texts_to_sequences(sentence)
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment = model.predict_classes(sentence,batch_size=1,verbose = 2)[0]
print(sentiment)
if sentiment == 1:
  print("Neutral")
elif sentiment == 0:
  print("Negative")
elif sentiment == 2:
  print("Positive")
else:
  print("Can not be determined")

2
Positive


In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs}
from sklearn.model_selection import GridSearchCV
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train, y=Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 36s - loss: 0.8306 - accuracy: 0.6451
Epoch 1/1
 - 36s - loss: 0.8196 - accuracy: 0.6464
Epoch 1/1


3 Q)

In [0]:
data1 = pd.read_csv('/content/spam.csv',encoding="ISO-8859-1")

# required columns
data1 = data1[['v1','v2']]
data1['v2'] = data1['v2'].apply(lambda x: x.lower())
data1['v2'] = data1['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data1.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data1['v2'].values)
X = tokenizer.texts_to_sequences(data1['v2'].values)

X = pad_sequences(X)

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data1['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [0]:
def createmodel1():
  embed_dim=128
  lstm_out=196
  model = Sequential()
  model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(2,activation='sigmoid'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model

In [27]:
batch_size = 32
model1 = createmodel1()
model1.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)
score,acc = model1.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model1.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 52s - loss: 0.1885 - accuracy: 0.9368
Epoch 2/5
 - 51s - loss: 0.0470 - accuracy: 0.9874
Epoch 3/5
 - 51s - loss: 0.0256 - accuracy: 0.9925
Epoch 4/5
 - 51s - loss: 0.0133 - accuracy: 0.9971
Epoch 5/5
 - 51s - loss: 0.0077 - accuracy: 0.9984
0.08270628494932203
0.9809679388999939
['loss', 'accuracy']
